# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_hf import *

# Load files and edit datasets

LAM output analysis.

Simu irr et simu no-irr.

In [ ]:
# lon_min=-10
# lon_max=4
# lat_min=34
# lat_max=46

In [ ]:
hf_dir='../../../JZ_simu_outputs/LAM/HF_outputs'

## LMDZ

In [ ]:
#noirr
filename = '{}/noirr/*_HF_histhf.nc'.format(hf_dir)

lmdz_noirr0 = xr.open_mfdataset(filename)
lmdz_noirr = lmdz_noirr0.rename({'time_counter':'time'})
lmdz_noirr = lmdz_noirr.assign_coords(time_decimal=lmdz_noirr.time.dt.hour + lmdz_noirr.time.dt.minute / 60)
lmdz_noirr.attrs['name'] = 'no_irr'
lmdz_noirr.attrs['plot_color']='red'
lmdz_noirr = lmdz_noirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

lmdz_noirr['sens']=-lmdz_noirr['sens']
lmdz_noirr['flat']=-lmdz_noirr['flat']

lmdz_noirr['ground_level'] = lmdz_noirr['phis'] / 9.81
lmdz_noirr['ground_level'].attrs['units'] = 'm'

lmdz_noirr

In [ ]:
#irr
filename = '{}/irr/*_HF_histhf.nc'.format(hf_dir)

lmdz_irr0 = xr.open_mfdataset(filename)
lmdz_irr =lmdz_irr0.rename({'time_counter':'time'})
lmdz_irr = lmdz_irr.assign_coords(time_decimal=lmdz_irr.time.dt.hour + lmdz_irr.time.dt.minute / 60)
lmdz_irr.attrs['name'] = 'irr'
lmdz_irr.attrs['plot_color']='#0C5DA5'
lmdz_irr = lmdz_irr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

lmdz_irr['sens']=-lmdz_irr['sens']
lmdz_irr['flat']=-lmdz_irr['flat']

lmdz_irr['ground_level'] = lmdz_irr['phis'] / 9.81
lmdz_irr['ground_level'].attrs['units'] = 'm'

lmdz_irr

## ORCHIDEE

In [ ]:
filename = '{}/noirr/*HF_sechiba_out_2.nc'.format(hf_dir)

orc_noirr0 = xr.open_mfdataset(filename)
orc_noirr = orc_noirr0.rename({'time_counter':'time'})
orc_noirr = orc_noirr.assign_coords(time_decimal=orc_noirr.time.dt.hour + orc_noirr.time.dt.minute / 60)
orc_noirr.attrs['name'] = 'no_irr'
orc_noirr.attrs['plot_color']='red'
orc_noirr = orc_noirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

orc_noirr

In [ ]:
filename = '{}/irr/*HF_sechiba_out_2.nc'.format(hf_dir)

orc_irr0 = xr.open_mfdataset(filename)
orc_irr = orc_irr0.rename({'time_counter':'time'})
orc_irr = orc_irr.assign_coords(time_decimal=orc_irr.time.dt.hour + orc_irr.time.dt.minute / 60)
orc_irr.attrs['name'] = 'irr'
orc_irr.attrs['plot_color']='#0C5DA5'
orc_irr = orc_irr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

orc_irr

## Masking

### IP continental subdomain

In [ ]:
# Define masks
 
con_mask_2D_ORC= orc_noirr.mean(dim='time')['Contfrac']>0.95
con_mask_2D_ORC.attrs['name']='Continental mask (95%)'

ip_polygon_mask_2D=polygon_to_mask(orc_noirr.mean(dim="time"), iberian_peninsula)
ip_polygon_mask_2D.attrs['name']='Iberian Peninsula polygon mask'

#combine two masks
ip_mask = con_mask_2D_ORC * ip_polygon_mask_2D
ip_mask.attrs['name']='IPmask'

In [ ]:
#ORC
ip_orc_noirr = apply_2Dmask_to_dataset(orc_noirr, ip_mask, dsname='no_irr')
ip_orc_irr = apply_2Dmask_to_dataset(orc_irr, ip_mask, dsname='irr')

In [ ]:
#LMDZ
ip_lmdz_noirr = apply_2Dmask_to_dataset(lmdz_noirr, ip_mask, dsname='no_irr')
ip_lmdz_irr = apply_2Dmask_to_dataset(lmdz_irr, ip_mask, dsname='irr')

## high_irr vs low_irr

In [ ]:
#on grid cell
#highirr
Cendrosa_lon=0.6
Cendrosa_lat=41.7
#lowirr
ElsPlans_lon=1.1
ElsPlans_lat=41.5

In [ ]:
cen_lmdz_noirr = lmdz_noirr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_lmdz_noirr.attrs['name']='La Cendrosa no_irr'
cen_lmdz_irr = lmdz_irr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_lmdz_irr.attrs['name']='La Cendrosa irr'
cen_orc_noirr = orc_noirr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_orc_noirr.attrs['name']='La Cendrosa no_irr'
cen_orc_irr = orc_irr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_orc_irr.attrs['name']='La Cendrosa irr'

In [ ]:
els_lmdz_noirr = lmdz_noirr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_lmdz_noirr.attrs['name']='Els Plans no_irr'
els_lmdz_irr = lmdz_irr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_lmdz_irr.attrs['name']='Els Plans irr'
els_orc_noirr = orc_noirr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_orc_noirr.attrs['name']='Els Plans no_irr'
els_orc_irr = orc_irr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_orc_irr.attrs['name']='Els Plans irr'

In [ ]:
cendrosa_gl= cen_lmdz_noirr['ground_level'].mean(dim='time').values.item()
elsplans_gl= els_lmdz_noirr['ground_level'].mean(dim='time').values.item()
cendrosa_gl,  elsplans_gl

# Maps

In [ ]:
var='sens'
ds=ip_lmdz_irr

vmin=0
vmax=1000
# vmin,vmax = None, None
cmap=myterrain
# title='off'
title=None
# title='Evaporative fraction (irr, JJA)'
# clabel='Irrigation (mm/d)'
clabel=None
map_ave(ds, var, vmin=vmin, vmax=vmax, cmap=cmap,
        title=title, clabel=clabel)

In [ ]:
#Diff map
var='t2m'
ds1=ip_lmdz_irr
ds2=ip_lmdz_noirr

vmax= 0.5
vmin=-0.5
# vmin,vmax = None, None
cmap=emb_neutral
title=None
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap, 
             sig=False,
             title=title, clabel=clabel)

In [ ]:
#Relative diff map
var='sens'
ds1=ip_lmdz_irr
ds2=ip_lmdz_noirr
maxvalue= 30
minvalue=-30
# minvalue, maxvalue = None, None

cmap=emb_neutral

map_rel_diff_ave(ds1, ds2, var, cmap=cmap, vmin=minvalue, vmax=maxvalue)

In [ ]:
#Display map of var for ds on restricted area
var='irrigation'
ds = ip_orc_irr
# ds = std
latmin=41.35
latmax=41.9
lonmin=0.6
lonmax=1.4
cmap='terrain'
vmin=0
vmax=500
title='Irrigation (July 2021, mm/d)'

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')

map_plotvar(plotvar, cmap=cmap, vmin=vmin,vmax=vmax, title=title)

# Time series and diurnal cycle

In [ ]:
var='phis'

ds1=lmdz_noirr
ds2=lmdz_irr
ds_list=[ds1, ds2]#, ds3]
# ds_list=[ds1]

title=None
time_series_ave(ds_list, var, title='{} ({})'.format(var, ds1[var].attrs['units']))


In [ ]:
#en un point precis
lon=Cendrosa_lon
lat=Cendrosa_lat

lon=ElsPlans_lon
lat=ElsPlans_lat
# lon=-2
# lat=41
var='ground_level'
ds1=lmdz_noirr
ds2=lmdz_irr
# ds1=orc_noirr
# ds2=orc_irr
ds_list=[ds1, ds2]
time_series_lonlat(ds_list, var, lon, lat)

In [ ]:
var='v10m'

ds1=ip_lmdz_noirr
ds2=ip_lmdz_irr
# ds1=ip_orc_noirr
# ds2=ip_orc_irr
ds_list=[ds1, ds2]#, ds3]
# ds_list=[ds2]

title=None
diurnal_cycle_ave(ds_list, var, title='{} ({})'.format(var, ds1[var].attrs['units']))

In [ ]:
var='sens'

ds1=ip_lmdz_noirr
ds2=ip_lmdz_irr
# ds1=ip_orc_noirr
# ds2=ip_orc_irr

ds_list=[ds1, ds2]#, ds3]
# ds_list=[ds2]

title=None

lon=Cendrosa_lon
lat=Cendrosa_lat

diurnal_cycle_lonlat(ds_list, var, lon=lon, lat=lat)

lon=ElsPlans_lon
lat=ElsPlans_lat

diurnal_cycle_lonlat(ds_list, var, lon=lon, lat=lat)

# Vertical profiles

In [ ]:
var='geoph'
ds1=els_lmdz_irr
ds2=els_lmdz_noirr
ds3=cen_lmdz_irr
ds4=cen_lmdz_noirr

# ds_list=[ds1, ds2, ds3, ds4]
ds_list=[ds1]

preslevelmax=2

title=None

profile_preslevs_local(ds_list, var, title=title, preslevelmax=preslevelmax)

In [ ]:
var='theta'
ds1=els_lmdz_irr
ds2=els_lmdz_noirr
ds3=cen_lmdz_irr
ds4=cen_lmdz_noirr

day='2017-07-10'

ds11=filter_xarray_by_day(ds1, day)
ds21=filter_xarray_by_day(ds2, day)
ds31=filter_xarray_by_day(ds3, day)
ds41=filter_xarray_by_day(ds4, day)

ds_list=[ds1, ds2, ds3, ds4]
ds_list=[ds11, ds21, ds31, ds41]

altmax=2500

title=None

# profile_preslevs_local(ds_list, var, title=title, preslevelmax=preslevelmax)
profile_altitude_local_mean(ds_list, var, title=title, altmax=altmax,
                       nbins=None,
                       substract_gl=False,
                       )

In [ ]:
ds11

In [ ]:
def profile_altitude_multipletimes(ds_list, var, times, altmin=0, altmax=2000):
    n_ax = len(times)
    fig, axs = plt.subplots(1, n_ax, figsize=(5*n_ax, 6))
    # Flatten axs only if it's an array (i.e., more than one subplot)
    axes = axs.flatten() if isinstance(axs, np.ndarray) else [axs]
    for i, time in enumerate(times):
        hour=times_correspondance[time]
        title = f"{var} at {hour}"
        
        # Filter datasets by the specified time and plot
        ds_list_tmp = [ds.where(ds['time_decimal']==time) for ds in ds_list]
        profile_altitude_local_mean(ds_list_tmp,
                                    var,
                                    ax=axes[i],
                                    title=title,
                                    altmin=altmin,
                                    altmax=altmax
        )

In [ ]:
var='theta'
ds1=els_lmdz_irr
ds2=els_lmdz_noirr
ds3=cen_lmdz_irr
ds4=cen_lmdz_noirr

ds_list=[ds1, ds2, ds3, ds4]

altmin=-0
altmax=2000

times=[1.5, 4.5, 7.5, 10.5, 13.5, 16.5, 19.5, 22.5]
times=[13.5, 16.5]
times=[13.5]

profile_altitude_multipletimes(ds_list, var, times, altmin=altmin, altmax=altmax)